# Mercedes-Benz Greener Manufacturing

Prediction on test df


In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
import itertools

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.decomposition import PCA

In [36]:
def lookups_for_one_hot(categorical_features_array, lookup_layers):

    list_values=[]
    for i in range(categorical_features_array.shape[1]):
        lookup = lookup_layers[i]
        np_to_tensor = tf.convert_to_tensor(categorical_features_array[:, i])
        onehot_encoded = lookup(np_to_tensor)
        list_values.append(onehot_encoded)

    return tf.concat(list_values, axis=1)

In [43]:
df = pd.read_csv("test/test.csv")

In [44]:
df.columns = [col.lower() for col in df]
ids = df[["id"]]
df.drop("id", axis=1, inplace=True)

In [45]:
df = df[df.select_dtypes(include="object").columns.to_list() + df.select_dtypes(exclude="object").columns.to_list()]

In [46]:
cols_to_drop = ['x11', 'x93', 'x107', 'x233', 'x235', 'x268', 'x289', 'x290', 'x293', 'x297', 'x330', 'x347']

In [47]:
df.drop(cols_to_drop, axis=1, inplace=True)
df = df.to_numpy()
df[:, :8] = df[:, :8].astype(str)

In [48]:
categorical_features = df[:, :8].astype(str) 
numerical_features = tf.convert_to_tensor(df[:, 8:].astype("float32"))

In [49]:
model = tf.keras.models.load_model("model")

In [69]:
lookup_layers = []
for i in range(8):
    with open(f"vocabularies/string_lookup_vocab_{i}.txt", "r") as f:
        loaded_vocab = [line.strip() for line in f]
    new_lookup_layer = tf.keras.layers.StringLookup(vocabulary=loaded_vocab, output_mode="one_hot")
    lookup_layers.append(new_lookup_layer)

In [71]:
test_categorical = lookups_for_one_hot(categorical_features, lookup_layers)

test_df = tf.concat([test_categorical, numerical_features], axis=1)

In [74]:
predictions = model.predict(test_df)

132/132 [==============================] - 0s 2ms/step


In [80]:
ids["y"] = predictions

In [83]:
ids.rename(columns={"id":"ID"}, inplace=True)

In [85]:
ids.to_csv("submission.csv", index=False)